# Stock Market Prediction Model of the Magnificient Seven 

#pip install the dependencies from requirements. May take up to 12 minutes

In [90]:
!python3 -m pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


### importing our libraries after pipinstall

In [91]:

#python data manipulation
import pandas as pd
import numpy as np

#data visualizaiton tools, EDA
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#mysql and Cassandra
import mysql.connector                         
from cassandra.cluster import Cluster          

#ARIMA
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

#LSTM Neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf

# our finance data
import yfinance as yf



## Extracting the Historical Data.


 ### The historical aspect of the data is starting from January 1st 2020 to yesterday (april 6th), We can adjust this to most recent always. We extracted the 7 companies and start with first 7 features from our project proposal: Date/time, open price, low price, high price, adjusted close price, trading volume

In [103]:
def extract_data(tickers, start_date="2020-01-01", end_date="2025-04-06", interval="1d"):
    # importing the yfinance api data, making sure the autoadjust is off because it overrides the close value if its on
    df = yf.download(tickers, start=start_date, end=end_date, interval=interval, auto_adjust=False)
    # tidying the data to make it better for analysis and transformation later
    df.columns = df.columns.swaplevel(0, 1)
    df = df.sort_index(axis=1, level=0)
    df_flat = df.stack(level=0, future_stack=True).reset_index()  
    df_flat.rename(columns={'level_0': 'Date'}, inplace=True)
    # renaming 'Adj Close' to 'Adj_Close' for consistency
    if 'Adj Close' in df_flat.columns:
        df_flat.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)
    # adding the previous day's close price per ticker
    df_flat['Previous_Close'] = df_flat.groupby('Ticker')['Close'].shift(1)
    # selecting the columns we expect and need
    expected_cols = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Previous_Close', 'Volume']
    existing_cols = [col for col in expected_cols if col in df_flat.columns]
    return df_flat[existing_cols]


In [102]:
#giving yfinance a list of companies to return a list of data of. AND displaying the df
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"]
df_raw = extract_data(tickers)
display(df_raw)

[*********************100%***********************]  7 of 7 completed


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume
0,2020-01-02,AAPL,74.059998,75.150002,73.797501,75.087502,72.716064,NaN,135480400
1,2020-01-02,AMZN,93.750000,94.900497,93.207497,94.900497,94.900497,NaN,80580000
2,2020-01-02,GOOGL,67.420502,68.433998,67.324501,68.433998,68.108368,NaN,27278000
3,2020-01-02,META,206.750000,209.789993,206.270004,209.779999,208.795944,NaN,12077100
4,2020-01-02,MSFT,158.779999,160.729996,158.330002,160.619995,153.323257,NaN,22622100
...,...,...,...,...,...,...,...,...,...
9249,2025-04-04,GOOGL,148.009995,151.070007,145.380005,145.600006,145.600006,150.720001,62138600
9250,2025-04-04,META,506.619995,518.000000,494.200012,504.730011,504.730011,531.619995,38515100
9251,2025-04-04,MSFT,364.130005,374.589996,359.480011,359.839996,359.839996,373.109985,49138700
9252,2025-04-04,NVDA,98.910004,100.129997,92.110001,94.309998,94.309998,101.800003,529707700


## Historical Data: Data Cleaning: 
### We will clean the data by taking care of any null values. We can use back and forward filling if there is a value missing from a float/int value. Otherwise we will drop the value if its in the date or Ticker/ or use the average of the past few days for the column. Additionally if there are are any duplicate records for a company and a speciifc date, one should be kept while rest dropped.

In [104]:
def cleaning(df):
    return df

## Historical Data, Feature Engineering:

### We are going to be transforming the tidy dataframe by doing some feature engineering. We will create the following fields into our table by manipulating the prexisting data from df_cleaned: 
1. Simple Moving Average (SMA)
2. Exponential Moving Average (EMA)
3. Relative Strength Index (RSI)
4. Bollinger Bands
5. MACD (Moving Average Convergence Divergence)
6. On-Balance Volume (OBV)
7. Volatility (ATR - Average True Range)


In [105]:
def transform(df):
    return df

## Loading into MySQL with historical data 

## Extracting Real time data
### Using Finnhub API to get real time data of the stock market , filling the Cassandra database every 10 seconds until 60seconds is reached for the past minute of data

In [139]:
from datetime import datetime
import finnhub

#using finhub api, use api key
finnhub_client = finnhub.Client(api_key="cvhdlvhr01qrtb3o0350cvhdlvhr01qrtb3o035g")

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"]

def get_realtime_quotes():
    quotes = []
    for symbol in tickers:
        data = finnhub_client.quote(symbol)
        quotes.append({
            "Date": pd.to_datetime(data['t'], unit='s'),
            "Ticker": symbol,
            "Open": data['o'],
            "High": data['h'],
            "Low": data['l'],
            "Close": data['c'],           
            "Adj_Close": data['c'],       
            "Previous_Close": data['pc'],  
            "Volume": None               
        })
    return pd.DataFrame(quotes)


## Loading into Cassandra with Real time data

In [140]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['localhost'])
    session = cluster.connect()
    print("Cassandra connection established.")
except Exception as e:
    print("Connection error:", e)


# connecting to  the Cassandra session
cluster = Cluster(['localhost'])
session = cluster.connect()

# we are making a keyspace called "stock data" and inside the keyspace we have a table called real_time_quotes where we are pulling data realtime from the finnhub api
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS stock_data 
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace('stock_data')
#schema for the cassandra csql table
session.execute("""
    CREATE TABLE IF NOT EXISTS real_time_quotes (
    ticker TEXT,
    date TIMESTAMP,
    open DOUBLE,
    high DOUBLE,
    low DOUBLE,
    close DOUBLE,
    adj_close DOUBLE,
    volume BIGINT,
    previous_close DOUBLE,
    PRIMARY KEY (ticker, date)
);
""")


Cassandra connection established.


In [141]:
#from cassandra.query import PreparedStatement
import time
#inserting the values from the api and clearing the white space and storing the columns as ticker, date, open, high, low, close, adj_close, and volume
def insert_to_cassandra(df):
    query = """
            INSERT INTO real_time_quotes (ticker, date, open, high, low, close, adj_close, volume, previous_close)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """

    for _, row in df.iterrows():
        session.execute(query, (
            row['Ticker'],
            row['Date'].to_pydatetime(),
            row['Open'],
            row['High'],
            row['Low'],
            row['Close'],
            row['Adj_Close'],
            row['Volume'] if pd.notna(row['Volume']) else 0,
            row['Previous_Close'] if pd.notna(row['Previous_Close']) else None
        ))  

#making a backup incase the cassandra server acts up
#def backup_to_csv(df, path='realtime_backup.csv'):
    #header = not os.path.exists(path)
    #df.to_csv(path, mode='a', index=False, header=header)



In [143]:
# pulls stocks values every 10 seconds, 14 times for about 2 min 33 seconds + some buffer time 
for _ in range(14):
    df_tenseconds = get_realtime_quotes()
    insert_to_cassandra(df_tenseconds)
    #backup_to_csv(df_hour)
    print(" For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.")
    time.sleep(10)  # delay to avoid rate limiting with the API


 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.
 For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.


In [144]:
#turning the cassandra table into a python pandas dataframe for datamanipulation
query = "SELECT * FROM stock_data.real_time_quotes"
rows = session.execute(query)
df_cassandra = pd.DataFrame(rows)
display(df_cassandra)

,ticker,date,adj_close,close,high,low,open,previous_close,volume
0,META,2025-04-08 16:23:31,523.500,523.500,547.43,520.10,520.10,516.25,0
1,META,2025-04-08 16:23:45,523.500,523.500,547.43,520.10,520.10,516.25,0
2,META,2025-04-08 16:23:52,523.500,523.500,547.43,520.10,520.10,516.25,0
3,META,2025-04-08 16:24:06,523.000,523.000,547.43,520.10,520.10,516.25,0
4,META,2025-04-08 16:24:13,523.000,523.000,547.43,520.10,520.10,516.25,0
...,...,...,...,...,...,...,...,...,...
93,MSFT,2025-04-08 16:25:05,363.185,363.185,373.65,359.18,359.18,357.86,0
94,MSFT,2025-04-08 16:25:14,363.185,363.185,373.65,359.18,359.18,357.86,0
95,MSFT,2025-04-08 16:25:24,363.185,363.185,373.65,359.18,359.18,357.86,0
96,MSFT,2025-04-08 16:25:38,363.185,363.185,373.65,359.18,359.18,357.86,0


## Real Time Data: Data Cleaning 


In [145]:
def cleaning(df):
    # making sure the date column is only in date time format
    df['date'] = pd.to_datetime(df['date'])
    #dropping rows that are missing ticker or date since its the most integral portion
    df = df.dropna(subset=['ticker', 'date'])
    # filling in the missing values using forward fill, or backfill when available 
    float_cols = ['open', 'high', 'low', 'close', 'adj_close', 'volume', 'previous_close']
    for col in float_cols:
        if col in df.columns:
            df[col] = df[col].ffill().bfill()
    # if there is still any remaining numerical columns (float)we can just average fill it 
    for col in float_cols:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].mean())
    # making sure theres no duplicate 
    df = df.drop_duplicates(subset=['ticker', 'date'], keep='last')
    # reseting index if we drop
    df = df.reset_index(drop=True)
    return df


#applying the cleaning function on our previous tile: df_cassandra dataframe
df_cassandracleaned = cleaning(df_cassandra)
display(df_cassandracleaned)
print("\nNull Count for Real Time Data (Extracted from Cassandra):\n")
print(df_cassandracleaned.isnull().sum())

,ticker,date,adj_close,close,high,low,open,previous_close,volume
0,META,2025-04-08 16:23:31,523.500,523.500,547.43,520.10,520.10,516.25,0
1,META,2025-04-08 16:23:45,523.500,523.500,547.43,520.10,520.10,516.25,0
2,META,2025-04-08 16:23:52,523.500,523.500,547.43,520.10,520.10,516.25,0
3,META,2025-04-08 16:24:06,523.000,523.000,547.43,520.10,520.10,516.25,0
4,META,2025-04-08 16:24:13,523.000,523.000,547.43,520.10,520.10,516.25,0
...,...,...,...,...,...,...,...,...,...
93,MSFT,2025-04-08 16:25:05,363.185,363.185,373.65,359.18,359.18,357.86,0
94,MSFT,2025-04-08 16:25:14,363.185,363.185,373.65,359.18,359.18,357.86,0
95,MSFT,2025-04-08 16:25:24,363.185,363.185,373.65,359.18,359.18,357.86,0
96,MSFT,2025-04-08 16:25:38,363.185,363.185,373.65,359.18,359.18,357.86,0



Null Count for Real Time Data (Extracted from Cassandra):

ticker            0
date              0
adj_close         0
close             0
high              0
low               0
open              0
previous_close    0
volume            0
dtype: int64


### Note: Finnhub API does not provide real time volume in its free API model. Additionally accuracy of per second volume would be hard to predict. We would potentially need to drop this or exclude volume entirely from subsequent machine learning later

## Real Time Data: Feature Engineering
### Can only perform this on a weekday between 8am EST and 4pm EST. Otherwise we will not be able to get more than 1 iteration of results since stock market is closed realtime. 
1. Simple Moving Average (SMA)
2. Exponential Moving Average (EMA)
3. Relative Strength Index (RSI)
4. Bollinger Bands
5. MACD (Moving Average Convergence Divergence)
6. On-Balance Volume (OBV)
7. Volatility (ATR - Average True Range)


In [146]:
def add_technical_indicators(df):
    df = df.sort_values(by=['ticker', 'date'])
    
    # calculating Simple Moving Average (SMA)
    df['SMA_5'] = df.groupby('ticker')['close'].transform(lambda x: x.rolling(window=5).mean())

    # calculating Exponential Moving Average (EMA)
    df['EMA_5'] = df.groupby('ticker')['close'].transform(lambda x: x.ewm(span=5, adjust=False).mean())

    # calculating Relative Strength Index (RSI)
    def compute_rsi(series, period=5):
        delta = series.diff()
        gain = delta.where(delta > 0, 0.0)
        loss = -delta.where(delta < 0, 0.0)
        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))
    
    df['RSI_5'] = df.groupby('ticker')['close'].transform(lambda x: compute_rsi(x))

    # calculating Bollinger Bands
    df['BB_Middle'] = df.groupby('ticker')['close'].transform(lambda x: x.rolling(window=5).mean())
    df['BB_Std'] = df.groupby('ticker')['close'].transform(lambda x: x.rolling(window=5).std())
    df['BB_Upper'] = df['BB_Middle'] + 2 * df['BB_Std']
    df['BB_Lower'] = df['BB_Middle'] - 2 * df['BB_Std']


    # calculating MACD
    ema12 = df.groupby('ticker')['close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
    ema26 = df.groupby('ticker')['close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
    df['MACD'] = ema12 - ema26
    df['MACD_Signal'] = df.groupby('ticker')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())

    # calclulating On Balance Volume (OBV)
    def compute_obv(group):
        obv = [0]
        for i in range(1, len(group)):
            if group['close'].iloc[i] > group['close'].iloc[i - 1]:
                obv.append(obv[-1] + group['volume'].iloc[i])
            elif group['close'].iloc[i] < group['close'].iloc[i - 1]:
                obv.append(obv[-1] - group['volume'].iloc[i])
            else:
                obv.append(obv[-1])
        return pd.Series(obv, index=group.index)

    df['OBV'] = df.groupby('ticker')[['close', 'volume']].apply(compute_obv).reset_index(level=0, drop=True)

    # calculating Average True Range (ATR)
    df['prev_close'] = df.groupby('ticker')['close'].shift(1)
    df['high_low'] = df['high'] - df['low']
    df['high_prev_close'] = (df['high'] - df['prev_close']).abs()
    df['low_prev_close'] = (df['low'] - df['prev_close']).abs()
    df['TR'] = df[['high_low', 'high_prev_close', 'low_prev_close']].max(axis=1)
    df['ATR_5'] = df.groupby('ticker')['TR'].transform(lambda x: x.rolling(window=5).mean())

    # dropping the temporary columns we had 
    df.drop(columns=['prev_close', 'high_low', 'high_prev_close', 'low_prev_close', 'TR'], inplace=True)

    return df

df_cleaned_cass_features=add_technical_indicators(df_cassandracleaned)
display(df_cleaned_cass_features)


,ticker,date,adj_close,close,high,low,open,previous_close,volume,SMA_5,EMA_5,RSI_5,BB_Middle,BB_Std,BB_Upper,BB_Lower,MACD,MACD_Signal,OBV,ATR_5
28,AAPL,2025-04-08 16:23:32,182.4800,182.4800,190.335,182.450,183.70,181.46,0,NaN,182.480000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,NaN
29,AAPL,2025-04-08 16:23:45,182.4800,182.4800,190.335,182.450,183.70,181.46,0,NaN,182.480000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,NaN
30,AAPL,2025-04-08 16:23:54,182.4800,182.4800,190.335,182.450,183.70,181.46,0,NaN,182.480000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,NaN
31,AAPL,2025-04-08 16:24:06,182.6232,182.6232,190.335,182.405,183.70,181.46,0,NaN,182.527733,NaN,NaN,NaN,NaN,NaN,0.011423,0.002285,0,NaN
32,AAPL,2025-04-08 16:24:13,182.6232,182.6232,190.335,182.405,183.70,181.46,0,182.53728,182.559556,100.0,182.53728,0.078434,182.694148,182.380412,0.020243,0.005876,0,7.903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,TSLA,2025-04-08 16:25:06,237.4700,237.4700,250.440,236.510,236.51,233.29,0,237.60704,237.598171,0.0,237.60704,0.076608,237.760255,237.453825,-0.078813,-0.045676,0,13.930
24,TSLA,2025-04-08 16:25:16,237.4700,237.4700,250.440,236.510,236.51,233.29,0,237.57278,237.555447,0.0,237.57278,0.093825,237.760430,237.385130,-0.090452,-0.054632,0,13.930
25,TSLA,2025-04-08 16:25:25,237.4700,237.4700,250.440,236.510,236.51,233.29,0,237.53852,237.526965,0.0,237.53852,0.093825,237.726170,237.350870,-0.098539,-0.063413,0,13.930
26,TSLA,2025-04-08 16:25:38,237.4700,237.4700,250.440,236.510,236.51,233.29,0,237.50426,237.507976,0.0,237.50426,0.076608,237.657475,237.351045,-0.103752,-0.071481,0,13.930


## real time data: final cleaning 
Feature engineering introduced new features some of witch now contain nans such as SMA5, RSI5, Bollinger Bonds(BB upper, lower,std,middle), and ATR5. However OBV is 0 since it relies on volume which is not provided by 0. We would need to drop the volume and OBV column since it does not provide us data. However for: SMA_5, RSI_5, BB_Upper/Lower/Std/middle, ATR_5 they all have rollin windows of 5 values. So its suppose to have have NaN values in the first rows.

In [155]:
def final_clean_realtime(df):
    # first droppoing OBV and volume since it is not available in finnhub
    df = df.drop(columns=['volume', 'OBV'])

    #to remedy the remaining nulls we are applying forward fill 
    rolling_cols = ['SMA_5', 'EMA_5', 'RSI_5', 'BB_Middle', 'BB_Std', 'BB_Upper', 'BB_Lower', 'ATR_5']
    df[rolling_cols] = df[rolling_cols].fillna(method='ffill')
    #dropping the remaining nans that are not able to be forward filled
    df = df.dropna(subset=rolling_cols).reset_index(drop=True)

    return df

df_cass_final = final_clean_realtime(df_cleaned_cass_features)
display(df_cass_final)
print('Which columns still have null?:')
print(df_cass_final.isnull().sum())


/var/folders/94/_l0wmbt11w71fxhk3zvh0t580000gn/T/ipykernel_32296/1523576798.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[rolling_cols] = df[rolling_cols].fillna(method='ffill')


,ticker,date,adj_close,close,high,low,open,previous_close,SMA_5,EMA_5,RSI_5,BB_Middle,BB_Std,BB_Upper,BB_Lower,MACD,MACD_Signal,ATR_5
0,AAPL,2025-04-08 16:24:13,182.6232,182.6232,190.335,182.405,183.70,181.46,182.53728,182.559556,100.0,182.53728,0.078434,182.694148,182.380412,0.020243,0.005876,7.903
1,AAPL,2025-04-08 16:24:26,182.6232,182.6232,190.335,182.405,183.70,181.46,182.56592,182.580770,100.0,182.56592,0.078434,182.722788,182.409052,0.026922,0.010086,7.912
2,AAPL,2025-04-08 16:24:34,182.6232,182.6232,190.335,182.405,183.70,181.46,182.59456,182.594914,100.0,182.59456,0.064041,182.722642,182.466478,0.031849,0.014438,7.921
3,AAPL,2025-04-08 16:24:46,182.6232,182.6232,190.335,182.405,183.70,181.46,182.62320,182.604342,100.0,182.62320,0.000000,182.623200,182.623200,0.035345,0.018620,7.930
4,AAPL,2025-04-08 16:24:54,182.6232,182.6232,190.335,182.405,183.70,181.46,182.62320,182.610628,100.0,182.62320,0.000000,182.623200,182.623200,0.037682,0.022432,7.930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,TSLA,2025-04-08 16:25:06,237.4700,237.4700,250.440,236.510,236.51,233.29,237.60704,237.598171,0.0,237.60704,0.076608,237.760255,237.453825,-0.078813,-0.045676,13.930
90,TSLA,2025-04-08 16:25:16,237.4700,237.4700,250.440,236.510,236.51,233.29,237.57278,237.555447,0.0,237.57278,0.093825,237.760430,237.385130,-0.090452,-0.054632,13.930
91,TSLA,2025-04-08 16:25:25,237.4700,237.4700,250.440,236.510,236.51,233.29,237.53852,237.526965,0.0,237.53852,0.093825,237.726170,237.350870,-0.098539,-0.063413,13.930
92,TSLA,2025-04-08 16:25:38,237.4700,237.4700,250.440,236.510,236.51,233.29,237.50426,237.507976,0.0,237.50426,0.076608,237.657475,237.351045,-0.103752,-0.071481,13.930


Which columns still have null?:
ticker            0
date              0
adj_close         0
close             0
high              0
low               0
open              0
previous_close    0
SMA_5             0
EMA_5             0
RSI_5             0
BB_Middle         0
BB_Std            0
BB_Upper          0
BB_Lower          0
MACD              0
MACD_Signal       0
ATR_5             0
dtype: int64


## Real Time Data: Encoding 

## Exploratory Data Analysis

## Machine Learning
### ARIMA for MySQL (historical) and LSTM for Cassandra (real time)